## *The Remote Atomic Swap - Alice's code
#### 04.5 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-04

* Store unsigned/signed transactions in a file
* Send partial transactions via e-mail

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
Alice  = cred['Alice']
Bob    = cred['Bob']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn, SignedTransaction
import algosdk.error

import json
import pandas as pd
import base64

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()['last-round']

## The remote swap
* Simple Atomic Swap: Alice send 2 Algos to Bob, and Bob send 1 Algos to Alice.
    * Usually, we would swap ALGOs for an ASA, this is just to simplify the example
* They are not in the same room.
* They exchange transactions via Email

### Gettin the timing right
* Inspect the suggested_params
* Transaction valid between `first` and `last`
* sp automatically chooses
    * `first` to be `last-round`
    * Validity time window to be 1000 blocks

In [ ]:
# Inspect the suggested_params ... 
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))
print(algod_client.status()["last-round"])

##### How much time to collect all signatures?
* `suggested_params()` proposes Algorand maximum = 1000 blocks

In [ ]:
# in minutes (one round is 3 sec )
print( (sp.last - sp.first)*3/60 )

#### Extend the time for all signatues
* Start validity window later $\rightarrow$ end later

In [ ]:
sp.first = sp.first+10      # start 10 rounds later
sp.last = sp.last+10        # end 10 rounds later

### The transaction steps

#### Step 1: prepare transactions
* Alice prepares both transactions
* Alternatively a programmer could prepare the two transactions and send one to Alice and one to Bob

In [ ]:
amt_1 = int(2*1E6)
txn_1 = transaction.PaymentTxn(Alice["public"], sp, Bob["public"],  amt_1)

amt_2 = int(1*1E6)
txn_2 = transaction.PaymentTxn(Bob["public"], sp, Alice["public"],  amt_2)

#### Step 2: create and assign group id

In [ ]:
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

#### Step 3: Send transaction file to Bob
* Transaction `txn_2` is now ready and can be sent to Bob
* To be able to save it into a file, we need to `dictify` it

In [ ]:
import pickle
data = txn_2.dictify()
file = open("Bob_txn.txt", 'wb')
pickle.dump(data, file)
file.close()

#### Step 4: Now it is Bob's turn
* We can assume that they exchange files via email or a similar service
* Open the notebook `04.5b_WSC`
* There we load the file `Bob_txn.txt` and create a signed transaction `Bob_signed.txt`

#### Step 5: Retrieve Bob's signed transaction
Instead of defining it as 'algosdk.transaction.PaymentTxn.undictify(...)', we use 'algosdk.transaction.SignedTransaction.undictify(...)'

In [ ]:
file = open("Bob_signed.txt", 'rb')
data = pickle.load(file)
# To undictify, we need the SignedTransaction function
stxn_2 = SignedTransaction.undictify(data)

#### Step 6: Alice has to sign her transaction

In [ ]:
stxn_1 = txn_1.sign(Alice['private'])

#### Step 7: Alice collects everything and sends the transaction
* This part could also be taken over by a third party

In [ ]:
# Step 7.1: collect
signed_group =  [stxn_1, stxn_2]

# Step 7.2: send
txid = algod_client.send_transactions(signed_group)

# Step 7.3: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid) 

## Appendix: how safe is it to send the transactions via email?
* The "all or noting" nature of the transaction group is protected by the `group_id`
    * The `group_id` is a hash of **all** transactions
    * If one transaction changed, the `group_id` changes
* Individual transactions are additionally protected by their signature

#### Scenario 1: Bob modifies his transaction before signing
* He wants to pay less
* This will not work, because ...
    * Alice will notice before signing her transaction
    * Even if she did not notice, the `goup_id` would change and the transaction group would fail
    
#### Scenario 2: A third player modifies Bob's transaction before he signs it
* This will not work, because the `goup_id` would change and the transaction group would fail

#### Scenario 3: A third player modifies Bob's transaction after he signs it
* This will not work, because the Bob's signature is not valid any more

#### Scenario 4: Alice modifies her transaction after Bob has signed
* She wants to pay less
* This will not work, because  the `goup_id` would change and the transaction group would fail

#### Scenario 5: Alice tries to be extra smart
* She waits for Bob's signature
* She modifies her transaction to pay less
* She calculates a new `goup_id`
* She changes the `goup_id` in Bob's (signed) transaction
* This will not work, because Bob's signature is not valid any more

## *A simple example of how the `group_id` protects an atomic swap

In [ ]:
# Step 1: prepare
sp = algod_client.suggested_params()

amt_1 = int(2*1E6)
txn_1 = transaction.PaymentTxn(Alice["public"], sp, Bob["public"],  amt_1)

amt_2 = int(1*1E6)
txn_2 = transaction.PaymentTxn(Bob["public"], sp, Alice["public"],  amt_2)

gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid
print( base64.b32encode(gid).decode() )

In [ ]:
# Step 1b: Bob changes his mind and wants to pay less
txn_2.amt = int(0.1*1E6)

In [ ]:
# Step 2: sign (everyone signs his/her transaction)
stxn_1 = txn_1.sign(Alice['private'])
stxn_2 = txn_2.sign(Bob['private'])

In [ ]:
# Step 3: collect and send
signed_group =  [stxn_1, stxn_2]
try:
    txid = algod_client.send_transactions(signed_group)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                                  # print entire error message
    if ("transactionGroup: incomplete group" in str(err)):      # check for specific type of error
        print("Transaction group incomplete or modified")         
    txid = None

# Step 4: no need to wait for confirmation